In [1]:
import httplib2
import json
import sys
import codecs
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np

In [85]:
def getMakeIDTable(Makelist):    
    result = pd.DataFrame(columns = ['EligibleMake', 'Make_ID', 'Make_Name'])
    for make in Makelist:
        temp = getMakeID(make.lower())
        if len(temp)<1:
            result = result.append({'EligibleMake':make,'Make_ID':'No Data','Make_Name':'No Data'},ignore_index=True)
        else:
            result = result.append({'EligibleMake':make,'Make_ID':temp['Make_ID'].values[0],'Make_Name':temp['Make_Name'].values[0]},ignore_index=True)
    return result

In [2]:
dictionary = {460:'ford',452:'bmw',467 :'chevrolet', 448:'toyota'}
ME_df = pd.read_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\F and E Series update 12020219.xlsx', sheet_name = 'MasterEligible', dtype=str)
ME_df

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription
0,Acura,CL,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
1,Acura,CL,2000,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
2,Acura,CL,2001,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
3,Acura,CL,2002,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
4,Acura,CL,2003,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
5,Acura,CL,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
6,Acura,CL,2004,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
7,Acura,CSX,2005,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
8,Acura,CSX,2006,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
9,Acura,CSX,2007,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2


In [86]:
AllMake = list(ME_df.Make.unique())
display(len(AllMake))
AllMake

80

['Acura',
 'Alfa Romeo',
 'Aston Martin',
 'Audi',
 'Bentley',
 'BMW',
 'Bugatti',
 'Buick',
 'Cadiillac',
 'Cadillac',
 'Cherolet',
 'Chevrolet',
 'Chevy',
 'Chrysler',
 'Daewoo',
 'Dodge',
 'DODGE, DODGE',
 'Eagle',
 'Ferrari',
 'Fiat',
 'Ford',
 'FORD',
 'FORD, FORD',
 'General Motors',
 'GM',
 'GMC',
 'GMC, GMC',
 'Honda',
 'HONDA',
 'Hummer',
 'Hyundai',
 'Infiniti',
 'Infinity',
 'Isuzu',
 'Jaguar',
 'Jeep',
 'Kia',
 'Lamborghini',
 'Land Rover',
 'Lexus',
 'LEXUS',
 'Lincoln',
 'Lotus',
 'Maserati',
 'Maybach',
 'Mazda',
 'MAZDA',
 'McLaren',
 'Mercedes Benz',
 'Mercedes-Benz',
 'Mercury',
 'Mini',
 'Mistubishi',
 'Mitsubishi',
 'MITSUBISHI',
 'MITUBISHI',
 'Nissan',
 'NISSAN',
 'Oldsmobile',
 'Opel',
 'Plymouth',
 'Pontiac',
 'Porsche',
 'RAM',
 'Ram',
 'Rolls-Royce',
 'Saab',
 'Saturn',
 'Scion',
 'SCION',
 'Smart',
 'Subaru',
 'SUBARU',
 'Suzuki',
 'SUZUKI',
 'Tesla',
 'Toyota',
 'TOYOTA',
 'Volkswagen',
 'Volvo']

In [76]:
def getMakeID(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    MK_df = pd.DataFrame(result['Results'])
    result_df = MK_df[MK_df['Make_Name'].str.lower().str.contains(Make)]
    return result_df.iloc[:1,:]
temp = getMakeID('Cadiillac')
temp


,Make_ID,Make_Name


In [83]:
df_1 = getMakeIDTable(AllMake[:20])
df_2 = getMakeIDTable(AllMake[20:40])
df_3 = getMakeIDTable(AllMake[40:60])
df_4 = getMakeIDTable(AllMake[60:])
df_all = pd.concat([df_1,df_2,df_3,df_4])
df_all

,EligibleMake,Make_ID,Make_Name
0,Acura,475,Acura
1,Alfa Romeo,493,Alfa Romeo
2,Aston Martin,440,Aston Martin
3,Audi,582,Audi
4,Bentley,583,Bentley
5,BMW,452,BMW
6,Bugatti,454,Bugatti
7,Buick,468,Buick
8,Cadiillac,No Data,No Data
9,Cadillac,469,Cadillac


In [84]:
df_all.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\MakeID_lookup.xlsx',index = False)